# ๏ Partie 2: Model Building

2 algorithmes avec au moins 2 paramètres différents (ex: max_depth, n_estimators,….)
Expliquer en vulgarisant le fonctionnement de vos algorithmes et ses paramètres

### Importer nos librairies

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
%matplotlib inline 

### Importer notre dataset

In [2]:
df = pd.read_csv('./data/new_cancer.csv')
df.head()

,Unnamed: 0,Age,Gender,Air Pollution,Alcohol use,Dust Allergy,OccuPational Hazards,Genetic Risk,chronic Lung Disease,Balanced Diet,...,Fatigue,Weight Loss,Shortness of Breath,Wheezing,Swallowing Difficulty,Clubbing of Finger Nails,Frequent Cold,Dry Cough,Snoring,Level
0,0,33,1,2,4,5,4,3,2,2,...,3,4,2,2,3,1,2,3,4,0
1,1,17,1,3,1,5,3,4,2,2,...,1,3,7,8,6,2,1,7,2,1
2,2,35,1,4,5,6,5,5,4,6,...,8,7,9,2,1,4,6,7,2,2
3,3,37,1,7,7,7,7,6,7,7,...,4,2,3,1,4,5,6,7,5,2
4,4,46,1,6,8,7,7,7,6,7,...,3,2,4,1,4,2,4,2,3,2


In [3]:
# Drop Unnamed column
df = df.drop("Unnamed: 0",axis=1)
list(df.columns)

['Age',
 'Gender',
 'Air Pollution',
 'Alcohol use',
 'Dust Allergy',
 'OccuPational Hazards',
 'Genetic Risk',
 'chronic Lung Disease',
 'Balanced Diet',
 'Obesity',
 'Smoking',
 'Passive Smoker',
 'Chest Pain',
 'Coughing of Blood',
 'Fatigue',
 'Weight Loss',
 'Shortness of Breath',
 'Wheezing',
 'Swallowing Difficulty',
 'Clubbing of Finger Nails',
 'Frequent Cold',
 'Dry Cough',
 'Snoring',
 'Level']

## Choix de nos variables


### Lors de l'exploration de notre dataset dans le fichier analyseGraphique.ipynb
### nous avons trouvé quelques features plus intéréssantes que d'autres que nous allons utilisé pour créer notre model de prédiction 

In [4]:

# X = df[["Dust Allergy","OccuPational Hazards","Genetic Risk","Obesity","Coughing of Blood"]]
y = df["Level"]
X = df.drop('Level',axis=1)

### Split data

In [58]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state=0)

### Étant donné que notre dataset ne contient que 1000 lignes je commence par tester le model LinearSVC

## Créer notre model LinearSVC

In [59]:
from sklearn.svm import LinearSVC

In [60]:
# Instancier la class LinearSVC
clf = LinearSVC()

# Entrainer notre modéle
clf.fit(X_train, y_train)

/home/djila/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


LinearSVC()

In [61]:
clf.score(X_test,y_test)

0.98

### On remarque ici que notre model à un excelent score de 98%
### Je vais donc faire de preprocessing à mon dataset pour voir si je peux amélioré le score 

In [62]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
clf_sc = make_pipeline(StandardScaler(),LinearSVC())
clf_sc.fit(X_train,y_train)
clf_sc.score(X_test,y_test)

0.9966666666666667

#### Après avoir fait le préprocessing avec StanderScaler j'obtiens un score de 99.66%, Excellente Résultats  

### Tester un autre model pour voir si nous obtiendrons un meilleur score
### Étant donné que mon dataset ne contient pas du text data Je vais tester le KNeighborsClassifier au lieu de Naive Bayes

In [65]:
from sklearn.neighbors import KNeighborsClassifier
kn_clf = KNeighborsClassifier(n_neighbors=6)
kn_clf.fit(X_train,y_train)
print("Le score est de {} avec le KNeighborsClassifier model".format(kn_clf.score(X_test,y_test)))

Le score est de 1.0 avec le KNeighborsClassifier model


### c'est impréssionnant j'obtients un score de 100% on scalant mes données 
### Je valide donc ce modèle pour déterminé le niveau de cancer chez un patient 

In [36]:
from sklearn.metrics import classification_report
preds = kn_clf.predict(X_test)
preds
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        99
           1       1.00      1.00      1.00        93
           2       1.00      1.00      1.00       108

    accuracy                           1.00       300
   macro avg       1.00      1.00      1.00       300
weighted avg       1.00      1.00      1.00       300



## mon modéle est trop précis car il prédit sans erreurs le niveau de cancer chez les 300 patients qu'il n'a jamais vu (auquel il ne s'est pas entrainé "donné de test")

### mon modéle obtient 100% de score je vais donc m'arrété là et je ne testerai pas le SVC ou l' EnsembleClassifiers